In [45]:
from pocketknife.database import (connect_database, read_from_database)
import pandas as pd
from datetime import datetime

In [20]:
id2legalform = {
    1: 'Sole proprietorship',  # Einzelunternehmen
    2: 'General Partnership',  # Kollektivgesellschaft
    3: 'Corporation',  # Aktiengesellschaft
    4: 'Limited Liability Company',  # Gesellschaft mit beschränkter Haftung
    5: 'Cooperative',  # Genossenschaft
    6: 'Association',  # Verein
    7: 'Foundation',  # Stiftung
    8: 'Public sector institution',  # Institut des öffentlichen Rechts
    9: 'Branch',  # Zweigniederlassung
    10: 'Limited Partnership',  # Kommanditgesellschaft
    11: 'Foreign branch',  # Zweigniederlassung einer ausl. Gesellschaft
    12: 'Corporation with unlimited partners',  # Kommanditaktiengesellschaft
    13: 'Special legal form',  # Besondere Rechtsform
    14: 'Ownership in undivided shares',  # Gemeinderschaft
    15: 'Limited Partnership for collective investment schemes with a fixed capital',  # Investmentgesellschaft mit festem Kapital
    16: 'Limited Partnership for collective investment schemes with a variable capital',  # Investmentgesellschaft mit variablem Kapital
    17: 'Limited Partnership for collective investment schemes',  # Kommanditgesellschaft für kollektive Kapitalanlagen
    18: 'Non commercial power of attorney',  # Nichtkaufmännische Prokure
    19: '(unknown)',  # (unbekannt)
}

growth_oriented_legal_forms = [2, 3, 4, 10, 12]

In [26]:
query = f"""
    SELECT 
        base.name,
        base.ehraid,
        base.uid,
        base.legal_seat_id,
        base.legal_seat,
        address.street,
        address.house_number,
        address.town,
        address.swiss_zip_code,
        address.country,
        base.legal_form_id,
        base.status, 
        base.delete_date,
        base.purpose_raw,
        shab.shab_id,
        shab.shab_date AS founding_date,
        shab.message AS founding_message
    FROM zefix.base base
    LEFT JOIN (
        SELECT s.ehraid, s.shab_id, s.shab_date, s.message
        FROM zefix.shab s
        WHERE s.shab_id IN (
            SELECT shab_id
            FROM zefix.shab_mutation
            WHERE description = 'status.neu'
        )
    ) AS shab
    ON base.ehraid = shab.ehraid
    LEFT JOIN zefix.address address
    ON base.ehraid = address.ehraid
    WHERE base.legal_form_id IN ({', '.join([str(id) for id in growth_oriented_legal_forms])});
"""

In [41]:
with connect_database() as con:
    df = read_from_database(connection=con, query=query)

In [46]:
df['founding_date'] = pd.to_datetime(df['founding_date'])
df['delete_date'] = pd.to_datetime(df['delete_date'])

df = df[(df['founding_date'] >= datetime(year=2016, month=1, day=1)) & (df['founding_date'] <= datetime(year=2024, month=12, day=31))]
df

,name,ehraid,uid,legal_seat_id,legal_seat,street,house_number,town,swiss_zip_code,country,legal_form_id,status,delete_date,purpose_raw,shab_id,founding_date,founding_message
825,Dow Orion Switzerland Holding GmbH,1313429,CHE198624070,1711,Zug,Dammstrasse,19,Zug,6300,CH,4,GELOESCHT,2018-06-29,"Erwerb, dauernde Verwaltung und Veräusserung v...",3.651143e+06,2017-07-18,Dow Orion Switzerland Holding GmbH (Dow Orion ...
836,Pacific Group SA,1252024,CHE252506093,5627,Chavannes-près-Renens,Chemin du Couchant,4,Chavannes-Renens,1022,CH,3,EXISTIEREND,NaT,la société a pour but toutes opérations immobi...,2.654123e+06,2016-02-08,"Pacific Group SA, à Chavannes-près-Renens, che..."
838,"Petite Puce, Lang & Baumann",1252311,CHE291790696,351,Bern,Brunngasse,56,Bern,3011,CH,2,GELOESCHT,2020-06-17,"Verkauf von Kleidung, neu und aus zweiter Hand...",2.657381e+06,2016-02-10,"Petite Puce, Lang & Baumann, in Bern, CHE-291...."
839,Gara Vita GmbH in Liquidation,1255195,CHE335643134,1054,Ebikon,Eschenweg,14,Ebikon,6030,CH,4,GELOESCHT,2018-05-18,Beratung in Versicherungs- und Finanzierungsfr...,2.704331e+06,2016-03-04,"Gara Vita GmbH, in Ebikon, CHE-335.643.134, Es..."
840,Auto Distribution AK Sàrl en liquidation,1255453,CHE292043320,5892,Blonay - Saint-Légier,Chemin de Chaponeyres,2,St-Légier-La Chiésaz,1806,CH,4,GELOESCHT,2024-05-27,"la société a pour but l'achat, la vente et le ...",2.707981e+06,2016-03-07,"Auto Distribution AK Sàrl, à Vevey, rue de l'U..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904983,SwissMex InmoInvestments GmbH,1653391,CHE287095713,351,Bern,Sandrainstrasse,91,Bern,3007,CH,4,EXISTIEREND,NaT,Erbringung von Beratungsdienstleistungen im Be...,1.006111e+09,2024-08-21,SwissMex InmoInvestments GmbH (SwissMex InmoIn...
904985,Frette Swiss GmbH,1653670,CHE331440796,1711,Zug,Landis + Gyr-Strasse,1,Zug,6300,CH,4,EXISTIEREND,NaT,"Die Gesellschaft bezweckt die Herstellung, die...",1.006113e+09,2024-08-23,"Frette Swiss GmbH, in Zug, CHE-331.440.796, La..."
904991,LTQuantum 1 GmbH,1656985,CHE416746314,261,Zürich,Scheideggstrasse,73,Zürich,8038,CH,4,EXISTIEREND,NaT,Ausschliesslicher Zweck der Gesellschaft ist d...,1.006132e+09,2024-09-18,"LTQuantum 1 GmbH (LTQuantum 1 LLC), in Zürich,..."
904993,RSB Financial Services GmbH,1669093,CHE486339670,3232,Balgach,Bühlstrasse,23,Balgach,9436,CH,4,EXISTIEREND,NaT,Die Gesellschaft bezweckt die Erbringung von F...,1.006200e+09,2024-12-10,"RSB Financial Services GmbH, in Balgach, CHE-4..."


In [49]:
df[df.status == 'GELOESCHT']

,name,ehraid,uid,legal_seat_id,legal_seat,street,house_number,town,swiss_zip_code,country,legal_form_id,status,delete_date,purpose_raw,shab_id,founding_date,founding_message
825,Dow Orion Switzerland Holding GmbH,1313429,CHE198624070,1711,Zug,Dammstrasse,19,Zug,6300,CH,4,GELOESCHT,2018-06-29,"Erwerb, dauernde Verwaltung und Veräusserung v...",3.651143e+06,2017-07-18,Dow Orion Switzerland Holding GmbH (Dow Orion ...
838,"Petite Puce, Lang & Baumann",1252311,CHE291790696,351,Bern,Brunngasse,56,Bern,3011,CH,2,GELOESCHT,2020-06-17,"Verkauf von Kleidung, neu und aus zweiter Hand...",2.657381e+06,2016-02-10,"Petite Puce, Lang & Baumann, in Bern, CHE-291...."
839,Gara Vita GmbH in Liquidation,1255195,CHE335643134,1054,Ebikon,Eschenweg,14,Ebikon,6030,CH,4,GELOESCHT,2018-05-18,Beratung in Versicherungs- und Finanzierungsfr...,2.704331e+06,2016-03-04,"Gara Vita GmbH, in Ebikon, CHE-335.643.134, Es..."
840,Auto Distribution AK Sàrl en liquidation,1255453,CHE292043320,5892,Blonay - Saint-Légier,Chemin de Chaponeyres,2,St-Légier-La Chiésaz,1806,CH,4,GELOESCHT,2024-05-27,"la société a pour but l'achat, la vente et le ...",2.707981e+06,2016-03-07,"Auto Distribution AK Sàrl, à Vevey, rue de l'U..."
841,TOLLOT & C. SA in Liquidation,1257818,CHE441528795,3982,Disentis/Mustér,Via Sontga Catrina,12,Disentis/Mustér,7180,CH,3,GELOESCHT,2023-01-04,"Die Gesellschaft bezweckt die Beschaffung, das...",2.746125e+06,2016-03-29,"TOLLOT & C. SA, in Savosa, CHE-441.528.795, Vi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904622,BRD Vertriebs GmbH in Liquidation,1413444,CHE492310597,193,Fällanden,,,ohne Domizil-sans domicile,,CH,4,GELOESCHT,2024-07-16,Die Gesellschaft bezweckt den Handel mit Marke...,1.004756e+09,2019-11-11,"BRD Vertriebs GmbH, in Fällanden, CHE-492.310...."
904626,Société Immobilière Le Rolliet - Aire J SA,1418496,CHE443948151,6607,Bernex,chemin de Paris,5,Bernex,1233,CH,3,GELOESCHT,2021-06-28,mise en valeur des droits à bâtir rattachés à ...,1.004786e+09,2019-12-17,"Société Immobilière Le Rolliet - Aire J SA, à ..."
904756,Lio Sonnenberg GmbH in Liquidation,1470797,CHE303662943,261,Zürich,Oberer Heuelsteig,30,Zürich,8032,CH,4,GELOESCHT,2024-07-05,Die Gesellschaft bezweckt das Führen von Gastr...,1.005085e+09,2021-01-28,"Lio Sonnenberg GmbH, in Zürich, CHE-303.662.94..."
904768,B5 BauArt GmbH in Liquidation,1484659,CHE323790053,2885,Eptingen,Schmiedengasse,14,Eptingen,4458,CH,4,GELOESCHT,2024-08-30,Die Gesellschaft bezweckt den Betrieb eines Ar...,1.005165e+09,2021-04-30,"B5 BauArt GmbH, in Eptingen, CHE-323.790.053, ..."


In [36]:
df

,name,ehraid,uid,legal_seat_id,legal_seat,street,house_number,town,swiss_zip_code,country,legal_form_id,status,delete_date,purpose_raw,shab_id,founding_message
founding_date,,,,,,,,,,,,,,,,
NaT,AMAG Leasing AG,5659,CHE107994662,1702,Cham,Alte Steinhauserstrasse,12,Cham,6330,CH,3,EXISTIEREND,NaT,Kauf und Verkauf sowie Vermietung (Leasing) vo...,NaN,None
NaT,Abbühl Architektur + Planung AG,313,CHE103864523,404,Burgdorf,Jlcoweg,6,Burgdorf,3400,CH,3,EXISTIEREND,NaT,Die Gesellschaft bezweckt die Ausführung von A...,NaN,None
NaT,Aung Kyaw GmbH,905500,CHE114164967,3340,Rapperswil-Jona,Untere Bahnhofstrasse,16,Rapperswil,8640,CH,4,EXISTIEREND,NaT,"Verkauf von und Handel mit Lebensmitteln, Taba...",NaN,None
NaT,MY Gastro GmbH in Liquidation,905638,CHE114167090,2579,Hägendorf,,,ohne Domizil-sans domicile,,CH,4,GELOESCHT,2018-01-26,Betrieb eines Bistros mit Pizza- und Kebapkuri...,NaN,None
NaT,ABT Aktiengesellschaft für Bautechnologie in L...,585,,1711,Zug,None,None,None,None,CH,3,GELOESCHT,1993-10-05,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-22,clever - schulen und beraten GmbH,1524883,CHE477544053,352,Bolligen,Kirchstrasse,10,Bolligen,3065,CH,4,EXISTIEREND,NaT,Die clever - schulen und beraten GmbH bezweckt...,1.005411e+09,"clever - schulen und beraten GmbH, in Ittigen,..."
2022-02-23,KEGEMA Sàrl,1525073,CHE218639042,443,Saint-Imier,Rue du Temple,6,St-Imier,2610,CH,4,EXISTIEREND,NaT,La société a pour but: Exploitation de cafés-r...,1.005412e+09,"KEGEMA Sàrl, à Saint-Imier, CHE-218.639.042, R..."
2022-03-01,spixel GmbH,1526047,CHE394579336,218,Ellikon an der Thur,Uesslingerstrasse,19a,Ellikon an der Thur,8548,CH,4,EXISTIEREND,NaT,"Die Gesellschaft bezweckt die Entwicklung, Pro...",1.005416e+09,"spixel GmbH, in Ellikon an der Thur, CHE-394.5..."


In [51]:
df[df.status == 'EXISTIEREND'][:20]

,name,ehraid,uid,legal_seat_id,legal_seat,street,house_number,town,swiss_zip_code,country,legal_form_id,status,delete_date,purpose_raw,shab_id,founding_date,founding_message
836,Pacific Group SA,1252024,CHE252506093,5627,Chavannes-près-Renens,Chemin du Couchant,4,Chavannes-Renens,1022,CH,3,EXISTIEREND,NaT,la société a pour but toutes opérations immobi...,2654123.0,2016-02-08,"Pacific Group SA, à Chavannes-près-Renens, che..."
843,Dysli Holding AG,1258135,CHE219684938,351,Bern,Looslistrasse,25,Bern,3027,CH,3,EXISTIEREND,NaT,"Die Gesellschaft bezweckt den Kauf, Verkauf un...",2751097.0,2016-03-31,"Dysli Holding AG, in Bern, CHE-219.684.938, Lo..."
844,1A-QA-RA-Services GmbH,1316860,CHE400761905,2770,Muttenz,Pfaffenmattweg,12,Muttenz,4132,CH,4,EXISTIEREND,NaT,Die Gesellschaft bezweckt die Beratung im Bere...,3710563.0,2017-08-23,"1A-QA-RA-Services GmbH, in Muttenz, CHE-400.76..."
846,InduTek-IT Sàrl,1261368,CHE383625216,6022,Chamoson,Ruelle de Parmay,12,Mayens-de-Chamoson,1955,CH,4,EXISTIEREND,NaT,"conception, développement, commercialisation e...",2802975.0,2016-04-27,"InduTek-IT Sàrl, à Châtel-sur-Montsalvens, Rou..."
994,Claudine et Jorge Correia Sàrl,1263873,CHE227458456,6136,Martigny,Avenue des Chênes,73,Charrat,1906,CH,4,EXISTIEREND,NaT,exploitation d'immeubles agricoles et d'une en...,2845819.0,2016-05-23,"Claudine et Jorge Correia Sàrl, à Charrat, CHE..."
999,TBF Coaching GmbH,1267707,CHE227191448,3782,Celerina/Schlarigna,Vietta Sur Crasta,6,Celerina/Schlarigna,7505,CH,4,EXISTIEREND,NaT,Die Gesellschaft bezweckt Beratungs- und sonst...,2905807.0,2016-06-22,"TBF Coaching GmbH, in St. Moritz, CHE-227.191...."
1006,Swiss Blade AG,1272772,CHE379732292,1708,Steinhausen,Bahnhofstrasse,63,Steinhausen,6312,CH,3,EXISTIEREND,NaT,Die Gesellschaft bezweckt die Erbringung von L...,2985219.0,2016-08-03,Medisantos AG (Medisantos SA) (Medisantos Ltd)...
1009,in2 GmbH,1276042,CHE268136441,2601,Solothurn,Ritterquai,6,Solothurn,4500,CH,4,EXISTIEREND,NaT,Erbringung von Dienstleistungen im Bereich Rau...,3044807.0,2016-09-08,"in2 GmbH, in Bellach, CHE-268.136.441, Viadukt..."
1013,Längle Rebbedarf GmbH,1279192,CHE409434648,4401,Arbon,Lehgasse,10,Stachen,9320,CH,4,EXISTIEREND,NaT,Die Gesellschaft bezweckt den Handel mit Waren...,3097057.0,2016-10-07,"Längle Rebbedarf GmbH, in Arbon, CHE-409.434.6..."
1014,Planatsch AG,1280471,CHE276355761,3986,Tujetsch,Bergrestaurant Planatsch,,Rueras,7189,CH,3,EXISTIEREND,NaT,Der Betrieb des Pistenrestaurants Planatsch mi...,3118807.0,2016-10-20,"Planatsch AG, in Tujetsch, CHE-276.355.761, Be..."
